In [1]:
import numpy as np
import pandas as pd
from sklearn.neighbors import NearestNeighbors
from features import ensembleFeature
import itertools
from utils import metricsCal
from thundersvm import SVC
from sklearn.model_selection import KFold
import pymrmr

In [2]:
##将序列转化为词向量用到的函数
def fa_seq(filepath):
    f = open(filepath,'r')
    x = []
    for i in f:
        x.append(i)
    seq = []
    for i in range(len(x)):
        if i%2==1:
            seq.append(x[i][:-1])
    return seq

In [3]:
train_seq = fa_seq("data/Mouse/mouse_train.fasta")
test_seq = fa_seq("data/Mouse/mouse_indep.fasta")
trainPos_seq = train_seq[0:int(len(train_seq)/2)]
trainNeg_seq = train_seq[int(len(train_seq)/2):]
testPos_seq = test_seq[0:int(len(test_seq)/2)]
testNeg_seq = test_seq[int(len(test_seq)/2):]
trainLabel = np.append(np.ones(int(len(train_seq)/2)),np.zeros(int(len(train_seq)-len(train_seq)/2)),axis=0)
testLabel = np.append(np.ones(int(len(test_seq)/2)),np.zeros(int(len(test_seq)-len(test_seq)/2)),axis=0)

In [4]:
# 值
trainPos,trainNeg,testPos,testNeg = ensembleFeature.PSKP(trainPos_seq,trainNeg_seq,testPos_seq,testNeg_seq,1) #3代表PSTP，2代表PSDP，1代表PSNP
trainData_PSNP = np.vstack((trainPos,trainNeg))
testData_PSNP = np.vstack((testPos,testNeg))
trainPos,trainNeg,testPos,testNeg = ensembleFeature.PSKP(trainPos_seq,trainNeg_seq,testPos_seq,testNeg_seq,2) #3代表PSTP，2代表PSDP，1代表PSNP
trainData_PSDP = np.vstack((trainPos,trainNeg))
testData_PSDP = np.vstack((testPos,testNeg))
trainPos,trainNeg,testPos,testNeg = ensembleFeature.PSKP(trainPos_seq,trainNeg_seq,testPos_seq,testNeg_seq,3) #3代表PSTP，2代表PSDP，1代表PSNP
trainData_PSTP = np.vstack((trainPos,trainNeg))
testData_PSTP = np.vstack((testPos,testNeg))
trainData_PSKP = np.hstack((trainData_PSNP,trainData_PSDP,trainData_PSTP))
testData_PSKP = np.hstack((testData_PSNP,testData_PSDP,testData_PSTP))



trainPos,trainNeg,testPos,testNeg = ensembleFeature.PSCP(trainPos_seq,trainNeg_seq,testPos_seq,testNeg_seq,0)
trainData_PSCP_0 = np.vstack((trainPos,trainNeg))
testData_PSCP_0 = np.vstack((testPos,testNeg))
trainPos,trainNeg,testPos,testNeg = ensembleFeature.PSCP(trainPos_seq,trainNeg_seq,testPos_seq,testNeg_seq,1)
trainData_PSCP_1 = np.vstack((trainPos,trainNeg))
testData_PSCP_1 = np.vstack((testPos,testNeg))
trainPos,trainNeg,testPos,testNeg = ensembleFeature.PSCP(trainPos_seq,trainNeg_seq,testPos_seq,testNeg_seq,2)
trainData_PSCP_2 = np.vstack((trainPos,trainNeg))
testData_PSCP_2 = np.vstack((testPos,testNeg))

trainData_PSCP = np.hstack((trainData_PSCP_0,trainData_PSCP_1,trainData_PSCP_2))
testData_PSCP = np.hstack((testData_PSCP_0,testData_PSCP_1,testData_PSCP_2))

trainPos,trainNeg,testPos,testNeg = ensembleFeature.BPB(trainPos_seq,trainNeg_seq,testPos_seq,testNeg_seq)
trainData_BPB = np.vstack((trainPos,trainNeg))
testData_BPB = np.vstack((testPos,testNeg))

trainData_KNN = np.array(pd.read_csv("features/KNN_value_train.csv").iloc[:,1:])
testData_KNN = np.array(pd.read_csv("features/KNN_value_test.csv").iloc[:,1:])

trainData = np.hstack((trainData_PSKP,trainData_PSCP,trainData_BPB,trainData_KNN))
testData = np.hstack((testData_PSKP,testData_PSCP,testData_BPB,testData_KNN))
trainData_loc = trainData

In [10]:
trainData = trainData_loc.reshape(trainData_loc.shape[0],1,-1)

In [6]:
import torch
from torch import nn
from torch.autograd import Variable
import torch.nn.functional as F

def attention(query, key, value, mask=None, dropout=None):  # q,k,v: [batch, h, seq_len, d_k]
    d_k = query.size(-1)  # query的维度
    scores = torch.matmul(query, key.transpose(-2, -1)) / math.sqrt(d_k)  # 打分机制 [batch, h, seq_len, seq_len]
    p_atten = F.softmax(scores, dim=-1)  # 对最后一个维度归一化得分, [batch, h, seq_len, seq_len]
    if dropout is not None:
        p_atten = dropout(p_atten)
    return torch.matmul(p_atten, value), p_atten  # [batch, h, seq_len, d_k] 作矩阵的乘法

class PositionalEncoding(nn.Module):

    def __init__(self, dim1, dim2, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()

        #if dim % 2 != 0:
        #    raise ValueError("Cannot use sin/cos positional encoding with "
        #                     "odd dim (got dim={:d})".format(dim))

        """
        构建位置编码pe
        pe公式为：
        PE(pos,2i/2i+1) = sin/cos(pos/10000^{2i/d_{model}})
        """
        pe = torch.zeros(max_len, dim2)  # max_len 是解码器生成句子的最长的长度，假设是 10
        position = torch.arange(0, max_len).unsqueeze(1)
        div_term0 = torch.exp((torch.arange(0, dim2, 2, dtype=torch.float) * -(math.log(10000.0) / dim2)))
        div_term1 = torch.exp((torch.arange(1, dim2, 2, dtype=torch.float) * -(math.log(10000.0) / dim2)))
        
        pe[:, 0::2] = torch.sin(position.float() * div_term0)
        pe[:, 1::2] = torch.cos(position.float() * div_term1)
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)
        #self.drop_out = nn.Dropout(p=dropout)
        self.dim2 = dim2
        self.bm1 = nn.BatchNorm1d(dim1,eps=1e-05)

    def forward(self, emb, step=None):
        emb = emb * math.sqrt(self.dim2)
        if step is None:
            emb = torch.tensor(emb) + self.pe[:,:emb.shape[1]]
        else:
            emb = emb + self.pe[step]
        #emb = self.drop_out(emb)
        emb = self.bm1(emb.to(torch.float32))
        return emb
    
def clones(module, N):  #定义clones方法
    return nn.ModuleList([copy.deepcopy(module)
                          for _ in range(N)])  #让原来变量不影响,且克隆module N次

class SelfAttention(nn.Module):  #多头注意力机制

    def __init__(self,embedding_dim, dropout=0.1):
        super(SelfAttention, self).__init__()
        self.linears = clones(nn.Linear(embedding_dim, embedding_dim), 4)  #克隆四份Linear网络层
        self.dropout = nn.Dropout(p=dropout)  #定义Dropout层

    def forward(self,query,key,value,mask=None):  # q,k,v: [batch, seq_len, embedding_dim]
        nbatches = query.shape[0]  #批数量
        query, key, value = [
            l(x) for l, x in zip(self.linears,
                            (query.to(torch.float32),
                             key.to(torch.float32),
                             value.to(torch.float32)))
        ]  #获取zip的query,key,value权重矩阵
        attn, p_atten = attention(query,key,value,mask=mask,dropout=self.dropout)
        out = self.linears[-1](attn)  #得到最后一层线性层的输出
        return out,p_atten  #返回out结果

In [7]:
from sklearn.utils import shuffle
from utils import metricsCal
from torch.utils.data import DataLoader,TensorDataset
import copy
from torch.autograd import Variable
from sklearn.model_selection import KFold

def train(model,data,label,epoch,train_device,model_dir,batch_size):
    if os.path.exists(model_dir+'model.pt'):
        model_train = torch.load(model_dir+'/model.pt')
    else:
        model_train = model
    criterion = nn.BCELoss()
    optimizer = torch.optim.Adam(model_train.parameters(),lr=0.003)  #改变学习率
    dataX = torch.Tensor(data).clone().detach()
    label = torch.Tensor(label).clone().detach()
    train_data = TensorDataset(dataX, label)
    train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size)
    running_loss = 0.0
    model_train = model_train.to(train_device)
    for batch_idx,data in enumerate(train_loader,0):
        inputs,target = data
        #inputs = inputs.reshape(inputs.shape[0], 1, inputs.shape[1])
        inputs = inputs.to(train_device)
        target = target.to(train_device)
        target = target.reshape(target.shape[0],1)
        optimizer.zero_grad()
        outputs = model_train(inputs)
        loss = criterion(outputs,target)
        loss.backward()
        optimizer.step()
        #print(type(loss))
        running_loss += loss.item()
        if batch_idx == len(dataX)//batch_size:
            #print('[%d, %5d] epoch loss: %.3f' %(epoch+1,batch_idx+1,running_loss))
            print(running_loss)
    save_model(model_train,model_dir)
    model_train = torch.load(model_dir+'/model.pt')
    #device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")  # 选择设备
    th,_,_,_,_,_,_,_,_,_ = metricsCal.evaluate(model_train,train_loader,train_device)
    return running_loss,th

def test(data,label,best_auc,test_device,model_dir,batch_size,th):
    model_test = load_model(model_dir)
    #model_test.eval()
    #model_test.to(test_device)
    data = torch.Tensor(data).clone().detach()#torch.Tensor(data)
    label = torch.Tensor(label).clone().detach()#.requires_grad_(True)torch.Tensor(label)
    test_data = TensorDataset(data,label)
    test_loader = DataLoader(test_data, shuffle=False, batch_size=batch_size)
    #device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")  # 选择设备
    _,_,_,_,Sen,Spe, Acc, mcc, AUC = metricsCal.evaluate(model_test,test_loader,test_device,False,th)
    
    print('Accuracy on test set: %d %%' %Acc)
    print('Sensitivity on test set: %d %%' %Sen)
    print('Speciality on test set: %d %%' %Spe)
    print('MCC on test set: %.3f' %mcc)
    print('auc on test set: %.3f' %AUC)
    if(AUC > best_auc):
        torch.save(model_test,model_dir+'model_best.pt')
    return Sen,Spe,Acc, mcc, AUC

def independTest(data,label,test_device,model_dir,batch_size,th):
    model_test = load_bestModel(model_dir)
    data = torch.Tensor(data).clone().detach()#torch.Tensor(data)
    label = torch.Tensor(label).clone().detach()#.requires_grad_(True)torch.Tensor(label)
    test_data = TensorDataset(data,label)
    test_loader = DataLoader(test_data, shuffle=False, batch_size=batch_size)
    #device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")  # 选择设备
    _,_,_,_,Sen,Spe, Acc, mcc, AUC = metricsCal.evaluate(model_test,test_loader,test_device,False,th)
    print(Acc,mcc,AUC)
    print('Accuracy on test set: %d %%' %Acc)
    print('Sensitivity on test set: %d %%' %Sen)
    print('Speciality on test set: %d %%' %Spe)
    print('MCC on test set: %.3f' %mcc)
    print('auc on test set: %.3f' %AUC)
    return Acc, mcc, AUC

def load_model(model_dir):
    if os.path.exists(model_dir+'model.pt'):
        model_load = torch.load(model_dir+'model.pt')
    return model_load

def save_model(model_save,model_dir):
    if not os.path.exists(model_dir):
        os.makedirs(model_dir)
    torch.save(model_save, model_dir+'model.pt')

def load_bestModel(model_dir):
    if os.path.exists(model_dir+'model_best.pt'):
        model_load = torch.load(model_dir+'model_best.pt')#,map_location='cuda:0')
    return model_load

In [12]:
#CNN
class Model_CNN(nn.Module):
    def __init__(self, dim1, dim2, dropout=0.1):
        super(Model_CNN, self).__init__()
        
        self.conv1 = nn.Conv2d(1,3,kernel_size=5,stride=2,padding=2)
        self.conv2 = nn.Conv2d(3,3,kernel_size=5,stride=2,padding=2)
        #self.conv3 = nn.Conv2d(3,3,kernel_size=5,stride=2,padding=2)
        
        k1 = int((dim1+2*2-5)/2)+1
        k2 = int((dim2+2*2-5)/2)+1
        k1 = int((k1+2*2-5)/2)+1
        k2 = int((k2+2*2-5)/2)+1
        #k1 = int((k1+2*2-5)/2)+1
        #k2 = int((k2+2*2-5)/2)+1
        
        self.fn1 = nn.Linear(3*k1*k2,128)
        self.fn2 = nn.Linear(128,1)
        self.ac = nn.Sigmoid()
        self.bm1 = nn.BatchNorm1d(dim1)
        
        self.relu = nn.ReLU()#inplace=True)
        self.drop = nn.Dropout(p=dropout)

    def forward(self, x):
        
        x1 = x.contiguous().view(-1,1,x.shape[1],x.shape[2])
        x1 = self.conv1(x1)
        x1 = self.relu(x1)
        x1 = self.conv2(x1)
        x1 = self.relu(x1)
        #x1 = self.conv3(x1)
        #x1 = self.relu(x1)
        x1 = x1.contiguous().view(x1.shape[0],-1)
        x1 = self.fn1(x1)
        x1 = self.relu(x1)
        x1 = self.drop(x1)
        x1 = self.fn2(x1)
        out = self.ac(x1)
        return out

#LSTM
class Model_LSTM(nn.Module):
    def __init__(self, dim1, dim2, dropout=0.1):
        super(Model_LSTM, self).__init__()
        self.posi = PositionalEncoding(dim1,dim2,dropout)
        self.lstm = nn.LSTM(input_size=dim2,hidden_size=dim2,batch_first=True,bidirectional=False)
        self.bm1 = nn.BatchNorm1d(dim1)
        self.conv1 = nn.Conv2d(1,1,kernel_size=5,stride=2,padding=2)
        k1 = int((dim1+2*2-5)/2)+1
        k2 = int((dim2+2*2-5)/2)+1
        self.fn1 = nn.Linear(1*k1*k2,128)
        #self.fn1 = nn.Linear(dim1*dim2,128)
        self.fn2 = nn.Linear(128,1)
        self.ac = nn.Sigmoid()
        
        self.relu = nn.ReLU()#inplace=True)
        self.drop = nn.Dropout(p=dropout)

    def forward(self, x):
        #所有输入，如果只有一维，需要变成1*n维
        x1 = self.posi(x)
        x1,(h_n,c_n) = self.lstm(x1)
        x1 = self.relu(x1)
        x1 = self.bm1(x1)
        
        x1 = x1.contiguous().view(-1,1,x1.shape[1],x1.shape[2])
        x1 = self.conv1(x1)
        x1 = self.relu(x1)
        x1 = x1.contiguous().view(x1.shape[0],-1)
        x1 = self.fn1(x1)
        x1 = self.relu(x1)
        x1 = self.drop(x1)
        x1 = self.fn2(x1)
        out = self.ac(x1)
        return out

#BiLSTM
class Model_BiLSTM(nn.Module):
    def __init__(self, dim1, dim2, dropout=0.1):
        super(Model_BiLSTM, self).__init__()
        self.posi = PositionalEncoding(dim1,dim2,dropout)
        self.lstm = nn.LSTM(input_size=dim2,hidden_size=dim2,batch_first=True,bidirectional=True)
        self.bm1 = nn.BatchNorm1d(dim1)
        self.conv1 = nn.Conv2d(1,1,kernel_size=5,stride=2,padding=2)
        k1 = int((dim1+2*2-5)/2)+1
        k2 = int((dim2+2*2-5)/2)+1
        self.fn1 = nn.Linear(1*k1*k2,128)
        self.fn2 = nn.Linear(128,1)
        self.ac = nn.Sigmoid()
        
        self.relu = nn.ReLU()#inplace=True)
        self.drop = nn.Dropout(p=dropout)

    def forward(self, x):
        #所有输入，如果只有一维，需要变成1*n维
        x1 = self.posi(x)
        x1,(h_n,c_n) = self.lstm(x1)
        x1 = x1[:,:,0:339]+x1[:,:,339:678]
        x1 = self.relu(x1)
        x1 = self.bm1(x1)
        x1 = x1.contiguous().view(-1,1,x1.shape[1],x1.shape[2])
        x1 = self.conv1(x1)
        x1 = self.relu(x1)
        x1 = x1.contiguous().view(x1.shape[0],-1)
        x1 = self.fn1(x1)
        x1 = self.relu(x1)
        x1 = self.drop(x1)
        x1 = self.fn2(x1)
        out = self.ac(x1)
        return out
    
#Attention
class Model_Attention(nn.Module):
    def __init__(self, dim1, dim2, dropout=0.1):
        super(Model_Attention, self).__init__()
        self.posi = PositionalEncoding(dim1,dim2,dropout)
        self.self_A = SelfAttention(dim2)
        self.self_B = SelfAttention(dim1)
        self.bm1 = nn.BatchNorm1d(dim1)
        self.conv1 = nn.Conv2d(1,1,kernel_size=5,stride=2,padding=2)
        k1 = int((dim1+2*2-5)/2)+1
        k2 = int((dim2+2*2-5)/2)+1
        self.fn1 = nn.Linear(1*k1*k2,128)
        self.fn2 = nn.Linear(128,1)
        self.ac = nn.Sigmoid()
        self.bm1 = nn.BatchNorm1d(dim1)
        
        self.relu = nn.ReLU()#inplace=True)
        self.drop = nn.Dropout(p=dropout)

    def forward(self, x):
        #所有输入，如果只有一维，需要变成1*n维
        x1 = self.posi(x)
        if x1.shape[1] == 1:
            x1 = x1.view(x1.shape[0],x1.shape[2],x1.shape[1])
            x1,attn = self.self_B(x1,x1,x1)
        else:
            x1,attn = self.self_A(x1,x1,x1)
        if x1.shape[2]==1:
            x1 = x1.view(x1.shape[0],x1.shape[2],x1.shape[1])
        x1 = self.bm1(x1)
        x1 = self.relu(x1)
        
        x1 = x1.contiguous().view(-1,1,x1.shape[1],x1.shape[2])
        x1 = self.conv1(x1)
        x1 = self.relu(x1)
        x1 = x1.contiguous().view(x1.shape[0],-1)
        x1 = self.fn1(x1)
        x1 = self.relu(x1)
        x1 = self.drop(x1)
        x1 = self.fn2(x1)
        out = self.ac(x1)
        return out

#LSTM_Attention_CNN
class Model_LSTM_Attention_CNN(nn.Module):
    def __init__(self, dim1, dim2, dropout=0.1):
        super(Model_LSTM_Attention_CNN, self).__init__()
        self.posi = PositionalEncoding(dim1,dim2,dropout)
        self.self_A = SelfAttention(dim2)
        self.self_B = SelfAttention(dim1)
        self.lstm = nn.LSTM(input_size=dim2,hidden_size=dim2,batch_first=True,bidirectional=False)
        self.bm1 = nn.BatchNorm1d(dim1)
        self.conv1 = nn.Conv2d(1,1,kernel_size=5,stride=2,padding=2)
        k1 = int((dim1+2*2-5)/2)+1
        k2 = int((dim2+2*2-5)/2)+1
        self.fn1 = nn.Linear(1*k1*k2,128)
        self.fn2 = nn.Linear(128,1)
        self.ac = nn.Sigmoid()
        self.bm1 = nn.BatchNorm1d(dim1)
        
        self.relu = nn.ReLU()#inplace=True)
        self.drop = nn.Dropout(p=dropout)
        #self.encoder = nn.Sequential()

    def forward(self, x):
        
        #所有输入，如果只有一维，需要变成1*n维
        x1 = self.posi(x)
        x1,(h_n,c_n) = self.lstm(x1)
        x1 = self.relu(x1)
        if x1.shape[1] == 1:
            x1 = x1.view(x1.shape[0],x1.shape[2],x1.shape[1])
            x1,attn = self.self_B(x1,x1,x1)
        else:
            x1,attn = self.self_A(x1,x1,x1)
        if x1.shape[2]==1:
            x1 = x1.view(x1.shape[0],x1.shape[2],x1.shape[1])
        x1 = self.bm1(x1)
        x1 = self.relu(x1)
        
        x1 = x1.contiguous().view(-1,1,x1.shape[1],x1.shape[2])
        x1 = self.conv1(x1)
        x1 = self.relu(x1)
        x1 = x1.contiguous().view(x1.shape[0],-1)
        x1 = self.fn1(x1)
        x1 = self.relu(x1)
        x1 = self.drop(x1)
        x1 = self.fn2(x1)
        out = self.ac(x1)
        return out
    
#BiLSTM_Attention_CNN
class Model_BiLSTM_Attention_CNN(nn.Module):
    def __init__(self, dim1, dim2, dropout=0.1):
        super(Model_BiLSTM_Attention_CNN, self).__init__()
        self.posi = PositionalEncoding(dim1,dim2,dropout)
        self.self_A = SelfAttention(dim2)
        self.self_B = SelfAttention(dim1)
        self.lstm = nn.LSTM(input_size=dim2,hidden_size=dim2,batch_first=True,bidirectional=True)
        self.bm1 = nn.BatchNorm1d(dim1)
        self.conv1 = nn.Conv2d(1,1,kernel_size=5,stride=2,padding=2)
        k1 = int((dim1+2*2-5)/2)+1
        k2 = int((dim2+2*2-5)/2)+1
        self.fn1 = nn.Linear(1*k1*k2,128)
        self.fn2 = nn.Linear(128,1)
        self.ac = nn.Sigmoid()
        self.bm1 = nn.BatchNorm1d(dim1)
        
        self.relu = nn.ReLU()#inplace=True)
        self.drop = nn.Dropout(p=dropout)

    def forward(self, x):
        
        #所有输入，如果只有一维，需要变成1*n维
        x1 = self.posi(x)
        x1,(h_n,c_n) = self.lstm(x1)
        x1 = x1[:,:,0:339]+x1[:,:,339:678]
        x1 = self.relu(x1)
        if x1.shape[1] == 1:
            x1 = x1.view(x1.shape[0],x1.shape[2],x1.shape[1])
            x1,attn = self.self_B(x1,x1,x1)
        else:
            x1,attn = self.self_A(x1,x1,x1)
        if x1.shape[2]==1:
            x1 = x1.view(x1.shape[0],x1.shape[2],x1.shape[1])
        x1 = self.bm1(x1)
        x1 = self.relu(x1)
        
        x1 = x1.contiguous().view(-1,1,x1.shape[1],x1.shape[2])
        x1 = self.conv1(x1)
        x1 = self.relu(x1)
        x1 = x1.contiguous().view(x1.shape[0],-1)
        x1 = self.fn1(x1)
        x1 = self.relu(x1)
        x1 = self.drop(x1)
        x1 = self.fn2(x1)
        out = self.ac(x1)
        return out

In [67]:
import os
import math
#先做交叉验证，看看多少个epoch合适
rows = 10
row = -1
max_epochs = 150
max_patience = 25
batch_size = 1024
row_name = []
Loss = np.zeros(rows*max_epochs).reshape(rows,max_epochs)
Sen = np.zeros(rows*(max_epochs+1)).reshape(rows,(max_epochs+1))
Spe = np.zeros(rows*(max_epochs+1)).reshape(rows,(max_epochs+1))
Acc = np.zeros(rows*(max_epochs+1)).reshape(rows,(max_epochs+1))
Mcc = np.zeros(rows*(max_epochs+1)).reshape(rows,(max_epochs+1))
Auc = np.zeros(rows*(max_epochs+1)).reshape(rows,(max_epochs+1))
th_All = np.zeros(rows*(max_epochs+1)).reshape(rows,(max_epochs+1))

#然后全部all_in 一个模型出来
#%env CUDA_LAUNCH_BLOCKING=1
kf = KFold(10,True,0)
#第三重循环
for i,[train_index, test_index] in enumerate(kf.split(trainData)):

    row+=1
    X_train = trainData[train_index]
    X_test = trainData[test_index]
    Y_train = trainLabel[train_index]
    Y_test = trainLabel[test_index]
    best_auc = 0
    patience = 0
    model_dir = "Model/Model_CNN_loc/KFold_" + str(i)+"/"
    model = Model_CNN(1,339)
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")  #选择设备
    model = model.to(device)
    os.environ['CUDA_VISIBLE_DEVICES'] = '0'

    for j in range(max_epochs):
        runningLoss,th = train(model,X_train,Y_train,j,device,model_dir,batch_size)
        sen,spe, acc, mcc, auc= test(X_test,Y_test,best_auc,device,model_dir,batch_size,th)
        Loss[row][j] = runningLoss
        th_All[row][j] = th
        Sen[row][j] = sen
        Spe[row][j] = spe
        Acc[row][j] = acc
        Mcc[row][j] = mcc
        Auc[row][j] = auc                            
        if auc > best_auc:
            best_auc=auc
            patience=0
        else:
            patience+=1
            
        if patience>max_patience and best_auc!= 0.5:
            break

/data/xiayunpeng/anaconda3/envs/BERT/lib/python3.6/site-packages/sklearn/utils/validation.py:72: FutureWarning: Pass shuffle=True, random_state=0 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)


6.235223054885864
Accuracy on test set: 0 %
Sensitivity on test set: 0 %
Speciality on test set: 0 %
MCC on test set: 0.418
auc on test set: 0.787
6.212037622928619
Accuracy on test set: 0 %
Sensitivity on test set: 0 %
Speciality on test set: 0 %
MCC on test set: 0.427
auc on test set: 0.793
6.128828406333923
Accuracy on test set: 0 %
Sensitivity on test set: 0 %
Speciality on test set: 0 %
MCC on test set: 0.431
auc on test set: 0.795
6.025821924209595
Accuracy on test set: 0 %
Sensitivity on test set: 0 %
Speciality on test set: 0 %
MCC on test set: 0.434
auc on test set: 0.795
5.832175016403198
Accuracy on test set: 0 %
Sensitivity on test set: 0 %
Speciality on test set: 0 %
MCC on test set: 0.442
auc on test set: 0.796
5.479314565658569
Accuracy on test set: 0 %
Sensitivity on test set: 0 %
Speciality on test set: 0 %
MCC on test set: 0.433
auc on test set: 0.797
5.390050947666168
Accuracy on test set: 0 %
Sensitivity on test set: 0 %
Speciality on test set: 0 %
MCC on test set: 

4.719776511192322
Accuracy on test set: 0 %
Sensitivity on test set: 0 %
Speciality on test set: 0 %
MCC on test set: 0.434
auc on test set: 0.808
4.849785268306732
Accuracy on test set: 0 %
Sensitivity on test set: 0 %
Speciality on test set: 0 %
MCC on test set: 0.448
auc on test set: 0.809
4.619632989168167
Accuracy on test set: 0 %
Sensitivity on test set: 0 %
Speciality on test set: 0 %
MCC on test set: 0.451
auc on test set: 0.809
4.916424036026001
Accuracy on test set: 0 %
Sensitivity on test set: 0 %
Speciality on test set: 0 %
MCC on test set: 0.432
auc on test set: 0.809
4.7555869817733765
Accuracy on test set: 0 %
Sensitivity on test set: 0 %
Speciality on test set: 0 %
MCC on test set: 0.450
auc on test set: 0.809
4.779483139514923
Accuracy on test set: 0 %
Sensitivity on test set: 0 %
Speciality on test set: 0 %
MCC on test set: 0.441
auc on test set: 0.809
4.628581464290619
Accuracy on test set: 0 %
Sensitivity on test set: 0 %
Speciality on test set: 0 %
MCC on test set:

4.707380294799805
Accuracy on test set: 0 %
Sensitivity on test set: 0 %
Speciality on test set: 0 %
MCC on test set: 0.467
auc on test set: 0.825
4.389986097812653
Accuracy on test set: 0 %
Sensitivity on test set: 0 %
Speciality on test set: 0 %
MCC on test set: 0.469
auc on test set: 0.824
4.655189156532288
Accuracy on test set: 0 %
Sensitivity on test set: 0 %
Speciality on test set: 0 %
MCC on test set: 0.476
auc on test set: 0.825
4.339442193508148
Accuracy on test set: 0 %
Sensitivity on test set: 0 %
Speciality on test set: 0 %
MCC on test set: 0.476
auc on test set: 0.825
4.449169337749481
Accuracy on test set: 0 %
Sensitivity on test set: 0 %
Speciality on test set: 0 %
MCC on test set: 0.477
auc on test set: 0.826
4.521268129348755
Accuracy on test set: 0 %
Sensitivity on test set: 0 %
Speciality on test set: 0 %
MCC on test set: 0.468
auc on test set: 0.826
4.518288105726242
Accuracy on test set: 0 %
Sensitivity on test set: 0 %
Speciality on test set: 0 %
MCC on test set: 

5.031124711036682
Accuracy on test set: 0 %
Sensitivity on test set: 0 %
Speciality on test set: 0 %
MCC on test set: 0.480
auc on test set: 0.828
4.932578951120377
Accuracy on test set: 0 %
Sensitivity on test set: 0 %
Speciality on test set: 0 %
MCC on test set: 0.481
auc on test set: 0.828
4.709765702486038
Accuracy on test set: 0 %
Sensitivity on test set: 0 %
Speciality on test set: 0 %
MCC on test set: 0.470
auc on test set: 0.828
5.046880722045898
Accuracy on test set: 0 %
Sensitivity on test set: 0 %
Speciality on test set: 0 %
MCC on test set: 0.474
auc on test set: 0.828
4.9523226618766785
Accuracy on test set: 0 %
Sensitivity on test set: 0 %
Speciality on test set: 0 %
MCC on test set: 0.468
auc on test set: 0.828
5.013670861721039
Accuracy on test set: 0 %
Sensitivity on test set: 0 %
Speciality on test set: 0 %
MCC on test set: 0.487
auc on test set: 0.829
4.847303748130798
Accuracy on test set: 0 %
Sensitivity on test set: 0 %
Speciality on test set: 0 %
MCC on test set:

...

Accuracy on test set: 0 %
Sensitivity on test set: 0 %
Speciality on test set: 0 %
MCC on test set: 0.510
auc on test set: 0.824
4.742530792951584
Accuracy on test set: 0 %
Sensitivity on test set: 0 %
Speciality on test set: 0 %
MCC on test set: 0.507
auc on test set: 0.825
4.85039809346199
Accuracy on test set: 0 %
Sensitivity on test set: 0 %
Speciality on test set: 0 %
MCC on test set: 0.515
auc on test set: 0.824
4.878037244081497
Accuracy on test set: 0 %
Sensitivity on test set: 0 %
Speciality on test set: 0 %
MCC on test set: 0.514
auc on test set: 0.824
4.650558590888977
Accuracy on test set: 0 %
Sensitivity on test set: 0 %
Speciality on test set: 0 %
MCC on test set: 0.515
auc on test set: 0.825
4.804986000061035
Accuracy on test set: 0 %
Sensitivity on test set: 0 %
Speciality on test set: 0 %
MCC on test set: 0.514
auc on test set: 0.824
4.832950711250305
Accuracy on test set: 0 %
Sensitivity on test set: 0 %
Speciality on test set: 0 %
MCC on test set: 0.515
auc on test s

4.857664942741394
Accuracy on test set: 0 %
Sensitivity on test set: 0 %
Speciality on test set: 0 %
MCC on test set: 0.526
auc on test set: 0.834
4.552999138832092
Accuracy on test set: 0 %
Sensitivity on test set: 0 %
Speciality on test set: 0 %
MCC on test set: 0.521
auc on test set: 0.834
4.63876485824585
Accuracy on test set: 0 %
Sensitivity on test set: 0 %
Speciality on test set: 0 %
MCC on test set: 0.527
auc on test set: 0.834
4.601947486400604
Accuracy on test set: 0 %
Sensitivity on test set: 0 %
Speciality on test set: 0 %
MCC on test set: 0.527
auc on test set: 0.835
4.601964473724365
Accuracy on test set: 0 %
Sensitivity on test set: 0 %
Speciality on test set: 0 %
MCC on test set: 0.523
auc on test set: 0.834
4.686542391777039
Accuracy on test set: 0 %
Sensitivity on test set: 0 %
Speciality on test set: 0 %
MCC on test set: 0.527
auc on test set: 0.834
4.658959716558456
Accuracy on test set: 0 %
Sensitivity on test set: 0 %
Speciality on test set: 0 %
MCC on test set: 0

Accuracy on test set: 0 %
Sensitivity on test set: 0 %
Speciality on test set: 0 %
MCC on test set: 0.535
auc on test set: 0.843
4.58583003282547
Accuracy on test set: 0 %
Sensitivity on test set: 0 %
Speciality on test set: 0 %
MCC on test set: 0.535
auc on test set: 0.845
4.545898467302322
Accuracy on test set: 0 %
Sensitivity on test set: 0 %
Speciality on test set: 0 %
MCC on test set: 0.539
auc on test set: 0.844
4.6015453934669495
Accuracy on test set: 0 %
Sensitivity on test set: 0 %
Speciality on test set: 0 %
MCC on test set: 0.541
auc on test set: 0.845
4.63620999455452
Accuracy on test set: 0 %
Sensitivity on test set: 0 %
Speciality on test set: 0 %
MCC on test set: 0.530
auc on test set: 0.845
4.479711085557938
Accuracy on test set: 0 %
Sensitivity on test set: 0 %
Speciality on test set: 0 %
MCC on test set: 0.533
auc on test set: 0.844
6.226631939411163
Accuracy on test set: 0 %
Sensitivity on test set: 0 %
Speciality on test set: 0 %
MCC on test set: 0.459
auc on test s

4.678517818450928
Accuracy on test set: 0 %
Sensitivity on test set: 0 %
Speciality on test set: 0 %
MCC on test set: 0.484
auc on test set: 0.808
4.690170079469681
Accuracy on test set: 0 %
Sensitivity on test set: 0 %
Speciality on test set: 0 %
MCC on test set: 0.484
auc on test set: 0.808
4.7069485783576965
Accuracy on test set: 0 %
Sensitivity on test set: 0 %
Speciality on test set: 0 %
MCC on test set: 0.484
auc on test set: 0.808
4.634809523820877
Accuracy on test set: 0 %
Sensitivity on test set: 0 %
Speciality on test set: 0 %
MCC on test set: 0.483
auc on test set: 0.809
4.591999769210815
Accuracy on test set: 0 %
Sensitivity on test set: 0 %
Speciality on test set: 0 %
MCC on test set: 0.484
auc on test set: 0.808
4.737914502620697
Accuracy on test set: 0 %
Sensitivity on test set: 0 %
Speciality on test set: 0 %
MCC on test set: 0.486
auc on test set: 0.809
4.635188490152359
Accuracy on test set: 0 %
Sensitivity on test set: 0 %
Speciality on test set: 0 %
MCC on test set:

Accuracy on test set: 0 %
Sensitivity on test set: 0 %
Speciality on test set: 0 %
MCC on test set: 0.484
auc on test set: 0.818
4.287783294916153
Accuracy on test set: 0 %
Sensitivity on test set: 0 %
Speciality on test set: 0 %
MCC on test set: 0.483
auc on test set: 0.818
4.664008617401123
Accuracy on test set: 0 %
Sensitivity on test set: 0 %
Speciality on test set: 0 %
MCC on test set: 0.491
auc on test set: 0.819
4.58525812625885
Accuracy on test set: 0 %
Sensitivity on test set: 0 %
Speciality on test set: 0 %
MCC on test set: 0.489
auc on test set: 0.819
4.5712458193302155
Accuracy on test set: 0 %
Sensitivity on test set: 0 %
Speciality on test set: 0 %
MCC on test set: 0.486
auc on test set: 0.819
4.438059240579605
Accuracy on test set: 0 %
Sensitivity on test set: 0 %
Speciality on test set: 0 %
MCC on test set: 0.489
auc on test set: 0.820
4.656611859798431
Accuracy on test set: 0 %
Sensitivity on test set: 0 %
Speciality on test set: 0 %
MCC on test set: 0.496
auc on test 

In [69]:
np.mean(np.max(Auc,axis=1))

0.8419389875896279

In [38]:
Row_name = []
name = ["Loss","Thr","Sen","Spe","Acc","Mcc","Auc"]
for i in range(7):
    for j in range(10):
        Row_name.append(name[i]+"_KFold_"+str(j))

In [70]:
pd.DataFrame(np.hstack((np.array(Row_name).reshape(-1,1),np.vstack((Loss,th_All[:,:150],Sen[:,:150],Spe[:,:150],Acc[:,:150],Mcc[:,:150],Auc[:,:150]))))).to_csv("Model/loc_CNN.csv")#,header=["Name","Loss","Thr","Sen","Spe","Acc","Mcc","Auc"])